In [2]:
import torch

/home/atraore/Desktop/gpt-from-scratch/env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [3]:
with open('input.txt', 'r') as f:
    text = f.read()

In [4]:
# Creating the vocabulary.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# Make encoder and decoder.
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello world !'))
print(decode(encode('hello world !')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 1, 2]
hello world !


In [9]:
# Encode the whole dataset.
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [10]:
# Split train/valid
train_ratio = 0.9
n = int(len(data) * train_ratio)
train_data = data[:n]
val_data = data[n:] 